In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import experiment

In [2]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

pd.set_option('display.float_format', lambda x: '%.4f' % x)

base = pd.read_csv('additional_data/base.csv')

folder = 'scaled'
sets = experiment.load_imputed_sets(folder)


In [3]:
pd.set_option('display.max_columns', 4)
pd.set_option('display.max_rows', 6)
df = experiment.reset_base()
df = df.reset_index()
df = df.set_index(['Indicator Name', 'Country Name'])
df = df.unstack().T
df

Indicator Name     Access to clean fuels and technologies for cooking (% of population)  \
     Country Name                                                                         
1990 Afghanistan                                                 NaN                      
     Albania                                                     NaN                      
     Algeria                                                     NaN                      
...                                                              ...                      
2020 Yemen, Rep.                                                 NaN                      
     Zambia                                                      NaN                      
     Zimbabwe                                                    NaN                      

Indicator Name     Access to electricity (% of population)  ...  \
     Country Name                                           ...   
1990 Afghanistan                                       NaN  ...   
     Albania                                        0.6983  ...   
     Algeria                                           NaN  ...   
...                                                    ...  ...   
2020 Yemen, Rep.                                       NaN  ...   
     Zambia                                            NaN  ...   
     Zimbabwe                                          NaN  ...   

Indicator Name     Urban population growth (annual %)  \
     Country Name                                       
1990 Afghanistan                               1.2449   
     Albania                                   0.1088   
     Algeria                                   0.8546   
...                                               ...   
2020 Yemen, Rep.                               0.7831   
     Zambia                                    0.8800   
     Zimbabwe                                 -0.3591   

Indicator Name     Vulnerable employment, total (% of total employment) (modeled ILO estimate)  
     Country Name                                                                               
1990 Afghanistan                                                 NaN                            
     Albania                                                     NaN                            
     Algeria                                                     NaN                            
...                                                              ...                            
2020 Yemen, Rep.                                                 NaN                            
     Zambia                                                      NaN                            
     Zimbabwe                                                    NaN                            

[4898 rows x 165 columns]

In [4]:
#get indicators with upper boundary 100
bondary_hundred = pd.read_csv('additional_data/chosen_indicators.csv', sep=';')
bondary_hundred.dropna(subset=['percentage'], inplace=True)
bondary_hundred.drop('percentage', axis=1, inplace=True)

#get indicators 
temp = base.drop('Country Name', axis=1).set_index('Indicator Name')
bondary_zero = temp[temp < 0].dropna(how='all').index.unique()

table = []
for name, df in sets.items():
    #count missing
    temp = df.set_index(['Country Name', 'Indicator Name'])
    missing = temp.isna().sum().sum()
    
    #count violations of boundary hundred
    temp = temp.loc[temp.index.isin(bondary_hundred['indicators'], level='Indicator Name')]
    implausible = (temp>100).sum().sum()
    
    #count violations of boundary zero
    temp = df.set_index(['Country Name', 'Indicator Name'])
    temp = temp.loc[~temp.index.isin(bondary_zero, level='Indicator Name')]
    implausible = implausible + (temp<-0.00000001).sum().sum()
    #something weird going on
    
    table.append([name, implausible, missing])

In [5]:
table = pd.DataFrame(table, columns=['Datensatz', 'logische Fehler', 'NaN'])
table = table.set_index('Datensatz')
print(table.to_latex(label='tab:logic_missing', caption='Prozentwerte \>100 für Indikatoren die diese nicht zulassen.'))

\begin{table}
\centering
\caption{Prozentwerte \>100 für Indikatoren die diese nicht zulassen.}
\label{tab:logic_missing}
\begin{tabular}{lrr}
\toprule
{} &  logische Fehler &     NaN \\
Datensatz              &                  &         \\
\midrule
Backfill               &                0 &  135192 \\
Overall Mean           &                0 &       0 \\
Yearly Mean            &                0 &   52298 \\
Yearly Mean per Region &                0 &   58134 \\
Interpolate 3          &                0 &  134029 \\
Interpolate all        &                0 &  112422 \\
ICE 1                  &             2888 &       0 \\
ICE 2                  &            16084 &       0 \\
ICE 3                  &            15729 &       0 \\
MICE 1                 &             8730 &       0 \\
MICE 2                 &            18474 &       0 \\
KNN 1                  &             4797 &       0 \\
KNN 2                  &                0 &       0 \\
\bottomrule
\end{tabular}
\end{tab

C:\Users\mbfbr\AppData\Local\Temp\ipykernel_21456\4167002823.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table.to_latex(label='tab:logic_missing', caption='Prozentwerte \>100 für Indikatoren die diese nicht zulassen.'))
